# 오늘의 날씨

(1) 날씨 데이터 : `todays_forecast.xlsx`
(2) 아이콘 데이터 : 


In [5]:
# 맵 생성, 지역별 구분

import folium
m = folium.Map([35.815669,127.7865791], zoom_start=7)
m

In [ ]:
# 일기예보 지역 리스트
region = [
{"name" : "강릉", loc : [37.62585944357624,127.01847823823798]},
{"name" : "수원", loc : [37.54248103738589,126.84414659211494]},
{"name" : "부산", loc : [37.5447438346992,126.95223862043447]},
{"name" : "안동", loc : [37.47943673430032,126.98346178441342]},
{"name" : "광주", loc : [37.61432815661055,127.0415039221072]},
{"name" : "대구", loc : [37.570756251987575,127.00235901546226]},
{"name" : "대전", loc : [35.91885025656432,128.55034430109828]},
{"name" : "서울", loc : [37.47627910055104,126.63147671205866]},
{"name" : "울산", loc : [35.14814807640179,126.90555219336346]},
{"name" : "인천", loc : [35.148338888178,126.93203920504686]},
{"name" : "광주", loc : [37.63618838692431,127.21107884503209]},
{"name" : "전주", loc : [37.42889361993242,127.10232604117991]},
{"name" : "제주", loc : [37.418353650942116,127.12640523182601]},
{"name" : "예산", loc : [37.34532813934063,126.73627994929885]},
{"name" : "청주", loc : [37.84401249737699,127.06236893072152]},
]


# 마커 추가
for i in region:
    folium.Marker(i.get("loc"), popup=i.get("store"), tooltip=i.get("store")).add_to(m)
